<a href="https://colab.research.google.com/github/Lucas60229/Lucaswu/blob/master/ga_report_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://keyword-hero.com/documentation/finding-your-view-id-in-google-analyticshttps://keyword-hero.com/documentation/finding-your-view-id-in-google-analytics

In [ ]:
import pandas as pd
# import pygsheets
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials


SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'key/ga-service-account-key.json'
# VIEW_ID = 'ga:172557771' #在query explorer就可以看到
# SHEET_ID = ''

# For the full list of dimensions & metrics, check https://developers.google.com/analytics/devguides/reporting/core/dimsmets
# DIMENSIONS = ['ga:source','ga:medium']
# METRICS = ['ga:users','ga:sessions']

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics


def get_report(analytics):
  return analytics.reports().batchGet(
      body=qs_to_reportRequests(qs)
  ).execute()


def convert_to_dataframe(response):
    
  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = [i.get('name',{}) for i in columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])]
    finalRows = []
    

    for row in report.get('data', {}).get('rows', []):
      dimensions = row.get('dimensions', [])
      metrics = row.get('metrics', [])[0].get('values', {})
      rowObject = {}

      for header, dimension in zip(dimensionHeaders, dimensions):
        rowObject[header] = dimension
        
        
      for metricHeader, metric in zip(metricHeaders, metrics):
        rowObject[metricHeader] = metric

      finalRows.append(rowObject)
      
      
  dataFrameFormat = pd.DataFrame(finalRows)    
  return dataFrameFormat      
        
 
def export_to_sheets(df):
    gc = pygsheets.authorize(service_file='client_secrets.json')
    sht = gc.open_by_key(SHEET_ID)
    wks = sht.worksheet_by_title('Sheet1')
    wks.set_dataframe(df,'A1')


def main():
  analytics = initialize_analyticsreporting()
  response = get_report(analytics)
  df = convert_to_dataframe(response)   #df = pandas dataframe
#   export_to_sheets(df)                  #outputs to spreadsheet. comment to skip
  return df                             


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
main()

In [ ]:
#剖析qs
from urllib.parse import urlparse, parse_qs
url = "https://www.googleapis.com/analytics/v3/data/ga?ids=ga%3A172557771&start-date=2019-09-01&end-date=2019-09-17&metrics=ga%3AtransactionRevenue&dimensions=ga%3AsourceMedium%2Cga%3AtransactionId%2Cga%3Adimension2%2Cga%3Adate&sort=ga%3Adate&filters=ga%3Asourcemedium%3D%40facebook&segment=gaid%3A%3A-1&samplingLevel=DEFAULT&include-empty-rows=true&start-index=1&max-results=20000"


result = urlparse(url)
qs = parse_qs(result.query)
qs

In [ ]:
#產生report 所需的物件
def sort_filed_parser(s):
    if s.startswith("-"):
        return (s.replace("-",""),"DESCENDING")
    else:
        return (s,"ASCENDING")
        
def qs_to_reportRequests(qs):
    rr = {'reportRequests': [
        {
          'viewId': qs["ids"][0],
          'dateRanges': [{'startDate': qs["start-date"][0], 'endDate':  qs["end-date"][0]}],
          'metrics':  [{'expression':i} for i in qs["metrics"][0].split(",")] ,
          'dimensions': [{'name':j} for j in qs["dimensions"][0].split(",")] ,
          "samplingLevel":  "LARGE", #預設使用large，資料大時會變慢，可以改用DEFAULT或SMALL
          "pageSize": 100000
        }]}
    if 'sort' in qs:
        rr['reportRequests'][0]["orderBys"] = [{"fieldName":sort_filed_parser(k)[0], "sortOrder": sort_filed_parser(k)[1]} for k in  qs["sort"][0].split(",")]
    if 'filters' in qs:
        rr['reportRequests'][0]["filtersExpression"] = qs["filters"][0]
    return rr
qs_to_reportRequests(qs)
    

In [ ]:
ga-service-account-credentials@keywordtag.iam.gserviceaccount.com